# Uldir Guild Awards Analysis

<ul>
<li><a href="#collect">Collect Analysis Data</a></li>
<li><a href="#taloc">Taloc</a></li>
<li><a href="#mother">MOTHER</a></li>
<li><a href="#zekvoz">Zek'voz</a>
<li><a href="#vectis">Vectis</a>
<li><a href="#fetid">Fetid</a>
<li><a href="#zul">Zul</a>
<li><a href="#mythrax">Mythrax</a>
<li><a href="#ghuun">G'Huun</a>
<li><a href="#overall">Overall</a>
    <ul>
        <li><a href="#pots">Most Pots Used</a></li>
        <li><a href="#healthstone">Most Healthstones/Pots Used</a></li>
        <li><a href="#rankings">Rankings</a></li>
        <li><a href="#prog-parse">Highest Parse on Progression Kill</a></li>
    </ul></li>
</ul>

This notebook extracts information from the [Warcraft Logs](https://www.warcraftlogs.com/) API to produce data for the [Last Pull](https://tempestgorg.enjin.com/) Uldir guild awards.

## Setup

In [1]:
# Import all required libraries and modules
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import warcraft_logs_fn as wl
import log_analysis as la
import api_key
%matplotlib inline

In [2]:
# Import key
api_key = api_key.key

In [4]:
# Update player info
player_list = pd.read_csv('player_list.csv')
player_list

,player,primary_role,alt,tier_end,tier_start,start_date
0,Vaelyra,mdps,NaN,NaN,True,NaN
1,Metonymy,rdps,NaN,NaN,True,NaN
2,Brian,mdps,NaN,NaN,True,NaN
3,Petrol,rdps,NaN,NaN,True,NaN
4,Ayriea,rdps,NaN,NaN,True,NaN
5,Stradlock,rdps,NaN,NaN,True,NaN
6,Uuglei,mdps,NaN,NaN,True,NaN
7,Earrl,tank,NaN,NaN,True,NaN
8,Illestra,healer,NaN,NaN,True,NaN
9,Kamer,mdps,NaN,NaN,True,NaN


In [30]:
# Update changed names
elz_idx = player_list.query('player == "Grimbrondal"').index
player_list.at[elz_idx, 'player'] = 'Grombrindal'

# Test
player_list.query('player == "Grombrindal"')

,player,primary_role,alt,tier_end,tier_start,start_date
10,Grombrindal,mdps,Cagliostro,False,True,NaN


In [15]:
def add_alts(previous_name, new_name, player_list, update=False):
    idx = player_list[player_list.player == previous_name].index
    if update:
        player_list.at[idx, 'player'] = new_name
        player_list.at[idx, 'alt'] = previous_name
    else:
        player_list.at[idx, 'alt'] = new_name

In [17]:
# Add player alts
add_alts('Ayriea', 'Invinceble', player_list)
add_alts('Vaelyra', 'Noraela', player_list)
add_alts('Sumi', 'Brassi', player_list)

# Test
player_list

,player,primary_role,alt,tier_end,tier_start,start_date
0,Vaelyra,mdps,Noraela,NaN,True,NaN
1,Metonymy,rdps,NaN,NaN,True,NaN
2,Brian,mdps,NaN,NaN,True,NaN
3,Petrol,rdps,NaN,NaN,True,NaN
4,Ayriea,rdps,Invinceble,NaN,True,NaN
5,Stradlock,rdps,NaN,NaN,True,NaN
6,Uuglei,mdps,NaN,NaN,True,NaN
7,Earrl,tank,NaN,NaN,True,NaN
8,Illestra,healer,NaN,NaN,True,NaN
9,Kamer,mdps,NaN,NaN,True,NaN


Alts were correctly added.

In [19]:
# Add new players
new_players = pd.DataFrame([
    {'player': 'Kopyelock', 
     'primary_role': 'rdps', 
     'alt': np.nan,
     'tier_end': True,
     'tier_start': False,
     'start_date': 'Oct-16-18'}, 
    {'player': 'Velsummers', 
     'primary_role': 'healer', 
     'alt': np.nan,
     'tier_end': True,
     'tier_start': True,
     'start_date': np.nan}], 
    columns=['player', 'primary_role', 'alt', 
             'tier_end', 'tier_start', 'start_date'])

new_players

,player,primary_role,alt,tier_end,tier_start,start_date
0,Kopyelock,rdps,NaN,True,False,Oct-16-18
1,Velsummers,healer,NaN,True,True,NaN


In [22]:
# Add to player_list
player_list = pd.concat([player_list, new_players])

# Test
assert player_list.tail(2).player.isin(['Kopyelock', 'Velsummers']).sum() == 2

In [31]:
# Confirm end of tier players
end_of_tier = ['Vaelyra', 'Metonymy', 'Brian', 'Petrol', 'Ayriea', 
               'Stradlock', 'Uuglei', 'Earrl', 'Illestra', 'Grombrindal', 
               'Tawn', 'Draigo', 'Psychodruid', 'Sumi', 'Toy', 'Mythrose', 
               'Shadowbaine', 'Acturus', 'Velryssa', 'Jampers', 'Vampyric', 
               'Kopyelock', 'Velsummers']

player_list.tier_end = player_list.player.isin(end_of_tier)

# Test
assert player_list.tier_end.sum() == len(end_of_tier)

In [32]:
# Update saved player list
player_list.to_csv('player_list.csv', index=False)

In [33]:
# Collect guild info
guild_info = wl.GUILD_INFO

# Extract logs
log_info = wl.get_logs(api_key, guild_info, log_start=1536202744238)
wl.save_logs(log_info, api_key, guild_info, log_start=1536202744238)

# Create boss list
boss_list = ['Taloc', 
             'MOTHER', 
             'Fetid Devourer', 
             "Zek'voz", 
             'Vectis',
             'Zul', 
             'Mythrax', 
             "G'Huun"]

# Extract fight details and save
fight_df = wl.extract_fights(boss_list)
wl.create_master_list(log_info, fight_df)

Creating file vTWVFQfwcHztbnKL
Creating file JgqKCkP7p3nAd1Ha
Creating file GbryNmfD98zcH6Jn
Creating file Hg83cdfhJaLyp6rG
Creating file k3LJpfYV1Nv2t8zq
Creating file mPaDJCdt4rWQkA7V
Creating file PYArFRQJ9MCT6jXy
Creating file rTPDJGpVXFZwNkA7
Creating file 9t1JjYfvbZkyWCGP
Creating file ac1rQnYvZWRjypVD
Creating file rMPLdKwCcH78Yz9q
Creating file 1rnktHXvcTWD2JzL
Creating file 96XVRq3ZJ2pGk1YC
Creating file FGhKD9AmjVpRMYw2
Creating file jaqf6V3Q4vNmDFdR
Creating file d3P97v4DpLHmyZkW
Creating file 6JMd2cmzFk8Qa1gj
Creating file ArqaVzbYtW7BG2Cx
Creating file 81HkLmCMRdAg6rNJ
Creating file nxf6krQP8Bvty9V2
Creating file V8Nd6ntR4PZMmLQW
Creating file VkQfpqZHB419zDcr
Creating file GdR1r6pVwA9TWXbJ
Creating file gcfCXbY14qDyhZtv
Creating file agwcpfjKbLTHrd1W
Creating file nqKDNkYjg9hzTwL6
Creating file T7xMvwtyG9YaP6k3
Creating file DycbAK6f1PL7Wpkd
Creating file rVcMgwvmPfDHYZ1a
Creating file VJQh2mdxkYypc196

All files created.

Log ID 1rnktHXvcTWD2JzL done.
Log ID 4mzCD2x9hnqZ

In [34]:
# Check all players added
fight_df.player_name.unique()

array(['Petrol', 'Illestra', 'Tawn', 'Liavre', 'Shadowbaine', 'Metonymy',
       'Draigo', 'Psychodruid', 'Kopyelock', 'Mythrose', 'Brian', 'Sumi',
       'Vaelyra', 'Katanescence', 'Acturus', 'Toy', 'Ayriea', 'Jampers',
       'Earrl', 'Grombrindal', 'Velryssa', 'Vampyric', 'Cagliostro',
       'Dough', 'Stradlock', 'Uuglei', 'Kiwisoda', 'Noraela', 'Katheals',
       'Invinceble', 'Brassi', 'Humanraptor', 'Czef', 'Quasilloyd',
       'Oakinon', 'Heavenlylite'], dtype=object)

Missing two player adds: 
1. Kiwisoda as main
2. Katheals as alt

In [35]:
add_alts('Katanescence', 'Katheals', player_list)

new_players = pd.DataFrame([
    {'player': 'Kiwisoda', 
     'primary_role': 'rdps', 
     'alt': np.nan,
     'tier_end': True,
     'tier_start': False,
     'start_date': 'Oct-23-18'}], 
    columns=['player', 'primary_role', 'alt', 
             'tier_end', 'tier_start', 'start_date'])

player_list = pd.concat([player_list, new_players])

# Test
assert player_list.alt.count() == 5
assert player_list.shape[0] == 28

In [36]:
# Update player list
player_list.to_csv('player_list.csv', index=False)

<a id='collect'></a>
## Collect Analysis Data

In [6]:
# Import key and player list
api_key = api_key.key
player_list = pd.read_csv('player_list.csv')

# Collect guild info
guild_info = wl.GUILD_INFO

# Extract logs
log_info = wl.get_logs(api_key, guild_info, log_start=1536202744238)
wl.save_logs(log_info, api_key, guild_info, log_start=1536202744238)

# Create boss list
boss_list = ['Taloc', 
             'MOTHER', 
             'Fetid Devourer', 
             "Zek'voz", 
             'Vectis',
             'Zul', 
             'Mythrax', 
             "G'Huun"]

# Extract fight details and save
fight_df = wl.extract_fights(boss_list)
wl.create_master_list(log_info, fight_df)

master_list = pd.read_csv('master_list.csv')
master_list = master_list[master_list.difficulty == 5] # Get mythic only
master_list.head()


All files created.

Log ID 1rnktHXvcTWD2JzL done.
Log ID 4mzCD2x9hnqZb3KG done.
Log ID 4zpRXjQA3Fhg9KdJ done.
Log ID 6JMd2cmzFk8Qa1gj done.
Log ID 81HkLmCMRdAg6rNJ done.
Log ID 96XVRq3ZJ2pGk1YC done.
Log ID 9t1JjYfvbZkyWCGP done.
Log ID ac1rQnYvZWRjypVD done.
Log ID agwcpfjKbLTHrd1W done.
Log ID ArqaVzbYtW7BG2Cx done.
Log ID ArVhTvdPmLf9JHjR done.
Log ID bKHF3fdQCJqGZzPt done.
Log ID d3P97v4DpLHmyZkW done.
Log ID dgkHRTaMQGJ7Zm3C done.
Log ID dHMk9Czm4GWRF8Tx done.
Log ID dV3RWaQchfv9zT64 done.
Log ID DycbAK6f1PL7Wpkd done.
Log ID FGhKD9AmjVpRMYw2 done.
Log ID fK7dCTJBknH64Xpz done.
Log ID FQwrJhRb2dypT6PC done.
Log ID GbryNmfD98zcH6Jn done.
Log ID gcfCXbY14qDyhZtv done.
Log ID GdR1r6pVwA9TWXbJ done.
Log ID Hg83cdfhJaLyp6rG done.
Log ID HQBfXNr6kqhP4dm1 done.
Log ID jaqf6V3Q4vNmDFdR done.
Log ID Jd2kb7C9PncfqN1v done.
Log ID JgqKCkP7p3nAd1Ha done.
Log ID JTrPMkjvCH6NtR7x done.
Log ID k3LJpfYV1Nv2t8zq done.
Log ID kgfWrMTP93417Yja done.
Log ID kKrTnBRXgtW3aJHw done.
Log ID KmVAr4DMhcvF

,log_id,log_start,log_end,pull_id,pull_start,pull_end,boss_id,boss_name,difficulty,kill,player_name
0,1rnktHXvcTWD2JzL,1542337560501,1542344234987,1,59280,241585,2135,Mythrax,5,False,Petrol
1,1rnktHXvcTWD2JzL,1542337560501,1542344234987,1,59280,241585,2135,Mythrax,5,False,Illestra
2,1rnktHXvcTWD2JzL,1542337560501,1542344234987,1,59280,241585,2135,Mythrax,5,False,Tawn
3,1rnktHXvcTWD2JzL,1542337560501,1542344234987,1,59280,241585,2135,Mythrax,5,False,Liavre
4,1rnktHXvcTWD2JzL,1542337560501,1542344234987,1,59280,241585,2135,Mythrax,5,False,Shadowbaine


<a id='taloc'></a>
## Taloc

Blood Storm: 270290

In [ ]:
# Damage done by Blood Storm
taloc = wl.damage_taken(api_key, lo)